In [1]:
import pandas as pd
import random
import numpy as np

## This code performs 2 algorithms, k-Means Clustering and Stepwise Forward Selection, on the Iris dataset from the UCI Machine Learning Repository https://archive.ics.uci.edu/ml/index.php

[Datasets](#Datasets):   
[Iris](#Iris)- 3 classes    
  
Clustering Algorithm:  
[k-Means](#k_means)   

Feature Selection Algorithm:  
[SFS](#SFS)    

[Models](#Models)  

<a id="Datasets"></a>
# Datasets

<a id="Iris"></a>
## Iris

In [2]:
iris = pd.read_csv("iris.data", header=None, names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width','iris_class'])

In [3]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
iris_class      150 non-null object
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


<a id="k_means"></a>
## k-Means

In [4]:
# k-Means is a unsupervised ML algorithm (no class labels are required to create the model) 
#         that identifies k clusters in the data
# The assumption for k-Means is that the underlying classes come from k Gaussian Distributions

# requires k > 0, data contains at least k records
# data is the full dataset
# features is a list with the index of each feature being used in the algorithm
# k is the number of clusters
# returns centroids and model_output
def k_means(data, features, k):
    np.random.seed(619884949)
    centroids = [] # centroids lists the k centroids
    n = len(data.index) # num of records in the data
    # initialize the random k centroids to start
    for index in range(k):
        # pull random record to use as initial starting point
        record_index = random.randint(1,n-1)
        random_record = data.iloc[[record_index]]
#        print(random_record)
        centroid = []
        for feature in features:
            centroid.append(random_record.iloc[0,feature])
        # end for
        centroids.append(centroid)
#        print(centroid)
    # end for
    # model_output is an array with n records, where the index matches the index of the records in the data, and the 
    # value matches the centroid associated with that record
    model_output = []
    convergence = 0
    cycles = 0
    while(convergence == 0):
        # update model_output with data based on current centroids
        model_output = calc_clusters(data, features, centroids)
#        print(centroids)
#        print(model_output)
        # recalculate centroids
        centroids, changes = calc_centroids(data, model_output, centroids, features)
        if(changes == 0):
            # no change to centroids leave while loop
            convergence = 1
        # end if
        cycles = cycles + 1
        if(cycles > 100):
            print('More than 100 cycles, timed out.')
            convergence = 1
        # end if
    # end while
    return centroids, model_output

In [5]:
# data is the full dataset
# features is a list with the index of each feature being used in the algorithm
# centroids is the current list of centroids
# returns model_output a list with n records, where the index matches the index of the records in the data, and the 
#      value matches the centroid associated with that record
def calc_clusters(data, features, centroids):
    model_output = []
    n = len(data.index) # num records in data
    # loop through all records
    for index in range(n):
        curr_record = data.iloc[[index]]
        shortest = 1000000000000 # default to very large number
        cluster = -1 # tracks the cluster for this record
        cluster_num = 0 # count to track which centroid we are testing for
        # loop through each centroid
        for centroid in centroids: # centroid is list of current means for each cluster
            # calc distance
            dist = distance(centroid, curr_record, features)
            if(dist < shortest): # if dist is less than shortest, this is the closest centroid so far
                cluster = cluster_num # assign this record to this centroid
                shortest = dist # assign new distance as closest
            # end if
            cluster_num = cluster_num + 1 # moving to next centroid
        # end for
        # after looping through all centroids, update the model_output with this index's assigned centroid
        model_output.append(cluster)
    # end for   
    
    return model_output

In [6]:
# data is the full dataset
# model_output a list with n records, where the index matches the index of the records in the data, and the 
#      value matches the centroid associated with that record
# centroids is the current list of centroids
# features is a list with the index of each feature being used in the algorithm
# returns new_centroids, which is the updated list of centroids, and changes which is the number of changes to the 
#        list of centroids provided to the function
def calc_centroids(data, model_output, centroids, features):
    new_centroids = [] # updates with new centroids based on model_output provided
    changes = 0 # tracks the changes to the centroids
#    print(centroids)
#    print(model_output)
    n = len(data.index) # num of records in the data
    num_features = len(features) # num of features
    centroid_number = 0 # tracks the centroid number being calculated
    for centroid in centroids: # centroid is the list of means
        new_centroid = [] # updates with new calculated means for this centroid
        for index in range(num_features): # index cycles through all features, the index of each feature, not the feature index
            new_feature_mean = 0 # tracks the calculation for the new mean
            records_in_centroid = 0.0001 # tracks the number of records for this centroid in the model_output. very small number in case there ends up with 0 records, no dividing by 0
            # loope through all records
            for record in range(n):
                curr_record = data.iloc[[record]]
                if(model_output[record]  == centroid_number): # if the model_output index for this record matches the current centroid being calculated
                    records_in_centroid = records_in_centroid + 1 # update the number of records for this centroid
                    new_feature_mean = new_feature_mean + curr_record.iloc[0,features[index]] # add the value of the feature for this record
                # end if
            # end for
            new_feature_mean = new_feature_mean / records_in_centroid # divide the total value of that feature for this centroid and divide by number of records
            new_centroid.append(new_feature_mean) # add new mean to this centroid
        # end for
        new_centroids.append(new_centroid) # add new centroid to the list of centroids
        # count how many of the centroids changed
        for count in range(num_features):
            if(centroid[count] != new_centroid[count]):
                changes = changes +1
            # end if
        # end for
        centroid_number = centroid_number + 1
    # end for
    return new_centroids, changes

In [7]:
# centroid is the centroid to calculate the distance to
# record is the instance to calculate the distance from
# features is a list with the index of each feature being used in the algorithm
# returns dist which is the distance from the centroid to the record
def distance(centroid, record, features):
    dist = 0
    # centroid is a list with the means for each feature
    # calculate distance between centroid and record
    count = 0 # keeps track of which # feature we are on
    for feature in features:
        dist = dist + (record.iloc[0,feature] - centroid[count])**2 # sum up all the squares
        count = count + 1
    # end for
    dist = np.sqrt(dist) # take the square root of the sum of squares
        
    return dist

<a id="SFS"></a>
## SFS

In [8]:
# Stepwise Forward Selection is a feature selection algorithm that begins by choosing one feature with the best performance,
# then adds a new feature if the performance is better than the prior best performance, on until there is no further
# improvement, at which point it returns that best performing list of features.

# Since this is a wrapper algorithm, this particular version utlizes the k-Means clustering algorithm above.

# features is a list with the index of all features being used in the algorithm 
# data is the full dataset
# k is the number of clusters
# returns f_0 which is the final selected list of features which has the best performance based on the SFS algorithm
def SFS(features, data, k):
    
    f_0 = [] # working feature set
    base_perf = -1000000 ## set to very small number
    f = features
    
    while(len(f) > 0):
        best_perf = -1000000 ## set to very small number-*
        best_F = 1
        # loop through all features
        for feature in f: # feature is the index of the current feature
            if(f_0 is None):
                f_0 = []
            # end if
            f_0.append(feature) # add current feature to working feature set
            print("new f being tested")
            print(f_0)
            centroids, model_output = k_means(data, f_0, k) # train new model_output with current feature set
            print(centroids)
            print(model_output)
            curr_perf = Perf(centroids, model_output, data, f_0) # determine the performance of the model
            print(curr_perf)
            if(curr_perf > best_perf): # if the current feature added increases the performance
                best_perf = curr_perf # update best_perf 
                best_F = feature 
            # end if
            f_0.remove(feature) # remove the feature from the current feature set
        # end for
        print(best_perf)
        print(best_F)
        print(base_perf)
        if(best_perf > base_perf): # check to see if the feature set created is better than the base
            print("best was better than base")
            print(f)
            print(best_F)
            base_perf = best_perf
            if(f_0 is None):
                f_0 = []
            # end if
            f_0.append(best_F)
            f.remove(best_F)
        # end if
        else:
            f = [] # empty F so it ends the while loop, the best F remaining isn't good enough to add to our feature set
        # end else
    # end while
    return f_0

In [9]:
# determines the performance of the model based on the Silhouette Coefficient


# centroids is the current list of centroids
# model_output a list with n records, where the index matches the index of the records in the data, and the 
#      value matches the centroid associated with that record
# data is the full dataset
# features is a list with the index of features being used
def Perf(centroids, model_output, data, features):
    final_perf = 0 # this will be the sum of all the coefficients for the data returned by this method
    n = len(data.index) # num records in data
    num_centroids = len(centroids) # num centroids (k)
    num_features = len(features) # number of features for this model
    
    # loop through all records to get s_i for each datapoint
    for index in range(n):
        curr_record = data.iloc[[index]]
        # calculate average distance to all other records, adding distance to variables according to centroid
        cluster_ci = model_output[index] # class predicted by the model
        sum_dist_cj = [0]*num_centroids # tracks the sum of all distances between record_xi and all points in the cluster_cj
        mi_points_in_cj = [0.0001]*num_centroids # total points in cluster_cj
        ave_dist_bi = 100000000000000 # set to very high number
        ave_dist_ai = 0 # default to 0
        for index_other in range(n):
            if(index_other != index):
                other_record = data.iloc[[index_other]]
                cluster_other = model_output[index_other]
                sum_dist_cj[cluster_other] = sum_dist_cj[cluster_other] + distance_perf(curr_record, other_record, features)
                mi_points_in_cj[cluster_other] = mi_points_in_cj[cluster_other] + 1
        # end for
        for index_centroid in range(num_centroids):
            if(index_centroid == cluster_ci):
                ave_dist_ai = sum_dist_cj[index_centroid] / mi_points_in_cj[index_centroid]
            # end if
            else:
                temp = sum_dist_cj[index_centroid] / mi_points_in_cj[index_centroid] 
                if(temp < ave_dist_bi):
                    ave_dist_bi = temp
                # end if
            # end else
            
        # then calculate the silhouette coefficient for record_xi
        s_i = (ave_dist_bi - ave_dist_ai) / max(ave_dist_ai, ave_dist_bi)
        final_perf = final_perf + s_i
    # end for
    # take the average of the silhouette coefficients
    final_perf = final_perf / n
    # return final performance measure
    return final_perf

In [10]:
def distance_perf(record1, record2, features):
    dist = 0
    # calculate distance between 2 records
    for feature in features: # feature is the index of the current feature
        dist = dist + ((record1.iloc[0,feature] - record2.iloc[0,feature])**2 )# sum up all the squares
    # end for
    dist = np.sqrt(dist) # take the square root of the sum of squares
        
    return dist

<a id="Models"></a>
## Models  

First we test the K-means algorithm with all features

In [11]:
iris_features = [0,1,2,3]
iris_means, iris_model_output = k_means(iris, iris_features, 3)

In [12]:
print(iris_means)

[[6.849981973731648, 3.073676121904941, 5.742090152394335, 2.071047181454785], [5.90160338451067, 2.7483826638989295, 4.393541300739837, 1.4338686550505564], [5.005989988020024, 3.4179931640136725, 1.463997072005856, 0.2439995120009759]]


In [13]:
print(iris_model_output)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]


In [14]:
iris_perf = Perf(iris_means, iris_model_output, iris, iris_features)

In [15]:
print(iris_perf)

0.5525919040451229


Next we test the SFS algorithm

In [16]:
iris_base_features = [0,1,2,3]
iris_SFS_features = SFS(iris_base_features, iris, 3)

new f being tested
[0]
[[5.769800434338803], [6.774496520595054], [4.8956415312140615]]
[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 0, 1, 0, 1, 2, 1, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0]
0.561274695778297
new f being tested
[1]
[[2.5851008827640785], [3.638878780892275], [3.068652136340095]]
[1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 0, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 1, 2, 0, 2, 0, 0, 2, 2, 1, 

In [17]:
print(iris_SFS_features)

[3]


In [18]:
iris_means, iris_model_output = k_means(iris, iris_SFS_features, 3)

In [19]:
print(iris_means)

[[2.058329045147822], [1.3230743787031178], [0.2439995120009759]]


In [20]:
print(iris_model_output)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
iris_perf = Perf(iris_means, iris_model_output, iris, iris_SFS_features)

In [22]:
print(iris_perf)

0.7259830692470867
